<a href="https://colab.research.google.com/github/murpunk/Programming_2023/blob/main/%D0%9F%D0%B0%D0%BD%D0%BA%D0%BE%D0%B2%D0%B0_4_NER_Natasha_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание

1. Выбрать 2 небольших текста для анализа (до 10000 знаков) на английском и русском языках.
2. Сделать их перевод с помощью любимого машинного переводчика
3. Для каждого текста на русском языке извлечь именованные сущности (Natasha) и построить их частотный словарь. Упрощенно сущности считаются одинаковыми, если у них совпадают first, last и middle
4. Сделать то же самое для текстов на английском (Spacy). Там можно брать
entity.text в качестве текста сущности
5. Сравните результаты. Есть ли отличия? Если да, что могло повляить на их появление?

Установка библиотек

In [ ]:
!pip install natasha

In [ ]:
!pip install spacy

Загружаю тексты

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Работа с Natasha

In [ ]:
from natasha import (
    Segmenter, MorphVocab,
    NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger,
    NamesExtractor, DatesExtractor, MoneyExtractor, AddrExtractor,
    PER, Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [ ]:
list(names_extractor(text))

In [ ]:
def extract_names(text):
    """Извлекает имена из текста"""
    doc = Doc(text)
    doc.segment(segmenter)  # token, sentence segmentation
    doc.tag_morph(morph_tagger)   # lemmatization
    # doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)  # Ner extractor
    for span in doc.spans:
        if span.type == PER:
            span.normalize(morph_vocab)  # приводим к начальной форме
            span.extract_fact(names_extractor)  # extract names
    names = []
    for span in doc.spans:
        if span.fact:
            pers_info = {'normal': span.normal,
                         'start': span.start,
                         'end': span.stop,
                         'tokens': [{'start': _.start, 'end': _.stop} for _ in span.tokens],
                         'text': span.text
                         }
            pers_info.update(span.fact.as_dict)
            names.append(pers_info)
    return names

In [ ]:
list(dates_extractor(text))

In [ ]:
list(money_extractor(text))

In [ ]:
list(addr_extractor(text))

Работа с spacy

In [ ]:
!python -m spacy download en_core_web_md # загрузка модели

In [5]:
import spacy           # подключение модели
import en_core_web_md

In [6]:
nlp = en_core_web_md.load() #Создаем pipeline